In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os
import pandas as pd

import numpy as np
import tifffile
from matplotlib import pyplot as plt
import fastplotlib as fpl

import caiman as cm
import mesmerize_core as mc
from mesmerize_core.caiman_extensions.cnmf import cnmf_cache
import lbm_caiman_python as lcp

WGPU: enumerate_adapters() is deprecated, use enumerate_adapters_sync() instead.
WGPU: request_adapter() is deprecated, use request_adapter_sync() instead.


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

Valid,Device,Type,Backend,Driver
✅ (default),NVIDIA RTX A4000,DiscreteGPU,Vulkan,560.94
✅,NVIDIA RTX A4000,DiscreteGPU,D3D12,
❗ limited,Microsoft Basic Render Driver,CPU,D3D12,
❌,NVIDIA RTX A4000/PCIe/SSE2,Unknown,OpenGL,4.6.0 NVIDIA 560.94


In [2]:
data_path = Path().home() / "lbm_data" / "demo_data"
mc.set_parent_raw_data_path(data_path)

WindowsPath('C:/Users/RBO/lbm_data/demo_data')

In [3]:
batch_path = data_path / 'batch.pickle'
df = mc.load_batch(batch_path)

In [4]:
df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid,min,max
0,mcorr,mcorr,plane_1.tiff,"{'main': {'pw_rigid': True, 'max_shifts': (6, 6), 'strides': (44, 44), 'overlaps': (13, 13), 'min_mov': None, 'gSig_...",{'mean-projection-path': e2258cfb-72eb-4781-816a-c018002b1759\e2258cfb-72eb-4781-816a-c018002b1759_mean_projection.n...,2025-01-11T19:23:44,2025-01-11T19:25:03,75.04 sec,None,e2258cfb-72eb-4781-816a-c018002b1759,NaN,NaN
1,mcorr,mcorr,plane_1.tiff,"{'main': {'pw_rigid': True, 'max_shifts': (2, 2), 'strides': (14, 14), 'overlaps': (4, 4), 'min_mov': None, 'gSig_fi...",{'mean-projection-path': 'af30391a-61b5-47bb-b7d4-ce5b6b668465/af30391a-61b5-47bb-b7d4-ce5b6b668465_mean_projection....,2025-01-11T19:33:33,2025-01-12T19:19:31,119.57 sec,None,af30391a-61b5-47bb-b7d4-ce5b6b668465,NaN,NaN
2,mcorr,mcorr,plane_1.tiff,"{'main': {'pw_rigid': True, 'max_shifts': (2, 2), 'strides': (14, 14), 'overlaps': (4, 4), 'min_mov': None, 'gSig_fi...",{'mean-projection-path': 'abb29c6c-7ab3-4f39-98d5-33bc19e05113/abb29c6c-7ab3-4f39-98d5-33bc19e05113_mean_projection....,2025-01-11T19:33:34,2025-01-12T19:21:31,120.22 sec,None,abb29c6c-7ab3-4f39-98d5-33bc19e05113,NaN,NaN
3,mcorr,mcorr,plane_1.tiff,"{'main': {'pw_rigid': True, 'max_shifts': (4, 4), 'strides': (29, 29), 'overlaps': (8, 8), 'min_mov': None, 'gSig_fi...",{'mean-projection-path': 'eea7041c-308a-4b78-8dec-d6b863feae4e/eea7041c-308a-4b78-8dec-d6b863feae4e_mean_projection....,2025-01-11T19:33:34,2025-01-12T19:22:55,83.48 sec,None,eea7041c-308a-4b78-8dec-d6b863feae4e,NaN,NaN
4,mcorr,mcorr,plane_1.tiff,"{'main': {'pw_rigid': True, 'max_shifts': (4, 4), 'strides': (29, 29), 'overlaps': (8, 8), 'min_mov': None, 'gSig_fi...",{'mean-projection-path': '2766892f-95f2-456b-8297-7709b91f0eb2/2766892f-95f2-456b-8297-7709b91f0eb2_mean_projection....,2025-01-11T19:33:35,2025-01-12T19:24:19,84.79 sec,None,2766892f-95f2-456b-8297-7709b91f0eb2,NaN,NaN
5,mcorr,mcorr,plane_1.tiff,"{'main': {'pw_rigid': True, 'max_shifts': (9, 9), 'strides': (58, 58), 'overlaps': (17, 17), 'min_mov': None, 'gSig_...",{'mean-projection-path': '3e684596-7809-4b54-9dab-fbf8f8ab2264/3e684596-7809-4b54-9dab-fbf8f8ab2264_mean_projection....,2025-01-11T19:33:35,2025-01-12T19:25:31,71.17 sec,None,3e684596-7809-4b54-9dab-fbf8f8ab2264,NaN,NaN
6,mcorr,mcorr,plane_1.tiff,"{'main': {'pw_rigid': True, 'max_shifts': (9, 9), 'strides': (58, 58), 'overlaps': (17, 17), 'min_mov': None, 'gSig_...",{'mean-projection-path': '48e84d2d-e29d-4fd2-a1a3-5e1887499994/48e84d2d-e29d-4fd2-a1a3-5e1887499994_mean_projection....,2025-01-11T19:33:35,2025-01-12T19:26:44,73.41 sec,None,48e84d2d-e29d-4fd2-a1a3-5e1887499994,NaN,NaN


In [5]:
ops = lcp.default_ops()

In [8]:
metadata = lcp.get_metadata(list(data_path.glob('*.tif*'))[0])
metadata

{'image_height': 2478,
 'image_width': 145,
 'num_pages': 51900,
 'ndim': 3,
 'dtype': 'uint16',
 'size': 18648189000,
 'shape': [51900, 2478, 145],
 'num_planes': 30,
 'num_rois': 4,
 'num_frames': 1730.0,
 'frame_rate': 9.60806,
 'fov': [150, 600],
 'pixel_resolution': [1.04, 1.0],
 'roi_width_px': 144,
 'roi_height_px': 600,
 'sample_format': 'int16',
 'num_lines_between_scanfields': 24,
 'center_xy': [-1.428571429, 0],
 'line_period': 4.15652e-05,
 'size_xy': [0.9523809524, 3.80952381],
 'objective_resolution': 157.5}